In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import os
import shutil
import zipfile
import scrapy
from scrapy.crawler import CrawlerProcess
import logging

In [8]:


# Disable log messages
logging.getLogger('scrapy').propagate = False
logging.getLogger('scrapy').setLevel(logging.WARNING)


class QuoteSpider(scrapy.Spider):
    name = 'crawler'
    start_urls = ['https://myxalandri.gr']
    processed_pages = 0  # A counter to keep track of processed pages

    def parse(self, response):
        urls = []  # Create an empty list to store URLs
        for r in response.css('a'):
            url = r.css('::attr(href)').get()
            if url:
                urls.append(response.urljoin(url))  # Append the full URL to the list

        # Print all URLs from the first page
        for url in urls:
            print(url)

        self.processed_pages += 1
        if self.processed_pages == 1:
            raise scrapy.exceptions.CloseSpider(reason="Finished processing the first page.")

# Running the spider
process = CrawlerProcess()
process.crawl(QuoteSpider)
process.start()



NameError: name 'respose' is not defined